In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [2]:
def current_sort_method(observed_coordinates, expected_coordinates):
    """
    This is the current method we are using
    It directly calls np.sort, which actually never changes the order of points, but only flip x-coord and y-coord if x is grater than y. 
    """
    observed_coordinates = np.sort(np.array(observed_coordinates))
    expected_coordinates = np.sort(np.array(expected_coordinates))
    return observed_coordinates, expected_coordinates


def sort_points_new_method(observed_coordinates, expected_coordinates):
    """
    This is one attemp to follow the original logic of sorting the points. Indeed it could change the order of points, but there is still issues, 
    which made me finally swtich to the method below. 
    """
    observed_coordinates = np.array(sorted(observed_coordinates, key=lambda x: (x[0], x[1])))
    expected_coordinates = np.array(sorted(expected_coordinates, key=lambda x: (x[0], x[1])))
    return observed_coordinates, expected_coordinates


def reorder_points_by_matching(observed_coordinates, expected_coordinates):
    """
    This is the final method I chose. It is not simply sorting the points according to its x- or y= coordinates, but trying to match expected 
    and observed coords based on their distance matrix.
    """
    expected_coordinates = np.array(expected_coordinates)
    observed_coordinates = np.array(observed_coordinates)
    dist_matrix = scipy.spatial.distance.cdist(expected_coordinates, observed_coordinates)
    expected_idx_list, observed_idx_list = scipy.optimize.linear_sum_assignment(dist_matrix)
    return observed_coordinates[observed_idx_list], expected_coordinates[expected_idx_list]


def compare_detection_coordinates(
    expected_coordinates: list[list[float]],
    observed_coordinates: list[list[float]],
    coverage_dim: float = 1,
) -> bool:
    """
    This is the same piece of code in test_main.py to compare observed and expected detection coordinates. I delete the part of sorting and 
    will do sort/reorder outside. 
    """

    
    if expected_coordinates.shape != observed_coordinates.shape:
        return False

    # As we have already compared the shape of the coordinates in the previous check
    # we can assume that if expected_detections is empty then observed_detections is empty too
    if len(expected_coordinates) == 0:
        return True

    elif len(expected_coordinates) > 0:
        for idx in range(len(expected_coordinates)):
            # We compute the error between the expected and observed detections
            # We are only concerned with the magnitude of the error in the 2D plane
            # projection from the sensor to the floor. Thus, we use the L2-norm.
            displacement_vector = np.array(expected_coordinates[idx]) - np.array(observed_coordinates[idx])
            error = np.linalg.norm(displacement_vector, ord=2)
            if error <= 0.15 * coverage_dim:
                return True
    return False

# Case 1

In this case, the coordinates of the two points for expected detections are exactly the same as those of the observed detections, only in a different order. However, it can be seen that the np.sort method currently used does not actually perform sorting, resulting in a returned result of False. The other two methods correctly returned a result of True.

In [3]:
expected = [[0.40309513, 0.80200477], [0.42328028, 0.44380363]]
observed = [[0.42328028, 0.44380363], [0.40309513, 0.80200477]]

expected, observed = np.array(expected), np.array(observed)

should_be = True

print(f"Should be {should_be}.\n  raw_expected={np.round(expected, 3).tolist()}\n  raw_observed={np.round(observed, 3).tolist()}")

observed_new, expected_new = current_sort_method(observed, expected)
print(f"\nThe sort method currently in use returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")

observed_new, expected_new = sort_points_new_method(observed, expected)
print(f"\nThe new sort method that really sort points returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")


observed_new, expected_new = reorder_points_by_matching(observed, expected)
print(f"\nThe matching-reorder method returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")

Should be True.
  raw_expected=[[0.403, 0.802], [0.423, 0.444]]
  raw_observed=[[0.423, 0.444], [0.403, 0.802]]

The sort method currently in use returns: False
  new_expected=[[0.403, 0.802], [0.423, 0.444]]
  new_observed=[[0.423, 0.444], [0.403, 0.802]]

The new sort method that really sort points returns: True
  new_expected=[[0.403, 0.802], [0.423, 0.444]]
  new_observed=[[0.403, 0.802], [0.423, 0.444]]

The matching-reorder method returns: True
  new_expected=[[0.403, 0.802], [0.423, 0.444]]
  new_observed=[[0.403, 0.802], [0.423, 0.444]]


# Case 2

This case only presents two observed points, and it can be seen that the x-coordinate of the first point is greater than its y-coordinate. Surprisingly, the method currently in use has swapped the two, which is clearly incorrect.

In [4]:
observed = [[0.375, 0.25], [0.36742424242424243, 0.8257575757575758]]

print(f"raw_observed={np.round(observed, 3).tolist()}")

observed_new, _ = current_sort_method(observed, observed)
print(f"\nThe sort method currently in use: \nnew_observed={np.round(observed_new, 3).tolist()}")

observed_new, _ = sort_points_new_method(observed, observed)
print(f"\nThe new sort method that really sort points returns: \nnew_observed={np.round(observed_new, 3).tolist()}")

raw_observed=[[0.375, 0.25], [0.367, 0.826]]

The sort method currently in use: 
new_observed=[[0.25, 0.375], [0.367, 0.826]]

The new sort method that really sort points returns: 
new_observed=[[0.367, 0.826], [0.375, 0.25]]


# Case 3

This case is quite special. I initially thought that as long as both the expected and observed points were correctly sorted, everything would be fail-safe. However, in this case, although the observed points were correctly sorted, the expected points were incorrectly sorted by their y-coordinates because their x-coordinates were exactly the same, leading to a mistakenly returned False. In fact, even if the x-coordinates of the two expected points were not equal, as long as the coordinates of the first point were slightly less than those of the second point, the same mistake would occur. This means that when the coordinates of two points are close to each other, some minor, normal random errors could lead to completely opposite results, which is unacceptable.   

On the other hand, surprisingly, the method currently being used has "stumbled into the correct outcome" through errors, which may explain why, in some cases, performance actually declined after this bug was fixed.   

Meanwhile, the final method I recommend, the matching method, is not affected by this because it employs a completely different logic, which should also be more reasonable.

In [5]:
expected = [[0.5, 0.1875], [0.5, 0.6875]]
observed = [[0.5492424242424243, 0.2424242424242424], [0.4557291666666667, 0.6875]]

expected, observed = np.array(expected), np.array(observed)

should_be = True

print(f"Should be {should_be}.\n  raw_expected={np.round(expected, 3).tolist()}\n  raw_observed={np.round(observed, 3).tolist()}")

observed_new, expected_new = current_sort_method(observed, expected)
print(f"\nThe sort method currently in use returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")

observed_new, expected_new = sort_points_new_method(observed, expected)
print(f"\nThe new sort method that really sort points returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")

observed_new, expected_new = reorder_points_by_matching(observed, expected)
print(f"\nThe matching-reorder method returns: {compare_detection_coordinates(observed_new, expected_new)}")
print(f"  new_expected={np.round(expected_new, 3).tolist()}\n  new_observed={np.round(observed_new, 3).tolist()}")

Should be True.
  raw_expected=[[0.5, 0.188], [0.5, 0.688]]
  raw_observed=[[0.549, 0.242], [0.456, 0.688]]

The sort method currently in use returns: True
  new_expected=[[0.188, 0.5], [0.5, 0.688]]
  new_observed=[[0.242, 0.549], [0.456, 0.688]]

The new sort method that really sort points returns: False
  new_expected=[[0.5, 0.188], [0.5, 0.688]]
  new_observed=[[0.456, 0.688], [0.549, 0.242]]

The matching-reorder method returns: True
  new_expected=[[0.5, 0.188], [0.5, 0.688]]
  new_observed=[[0.549, 0.242], [0.456, 0.688]]


# CPU Time Comparison

The suggested method (reordering by matching) is significantly more complicated than the other two methods, leading to a increase of CPU time. Luckily, such an increase is not too much and should be affordable

### Current Method

In [6]:
%%timeit

observed_new, expected_new = current_sort_method(observed, expected)

1.58 µs ± 7.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### New Method of Sorting Points

In [7]:
%%timeit

observed_new, expected_new = sort_points_new_method(observed, expected)

2.25 µs ± 8.67 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### New Method of Reordering Points by Matching

In [8]:
%%timeit

observed_new, expected_new = reorder_points_by_matching(observed, expected)

3.26 µs ± 11.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
